In [1]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [4]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob

os.makedirs('part2', exist_ok = True)
os.makedirs('part2-audio', exist_ok = True)

In [5]:
session = boto3.Session()

In [6]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [7]:
for f in tqdm(bucket.objects.filter(Prefix='PART2/DATA/CHANNEL0/SCRIPT')):
    filename = os.path.join('part2', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

2003it [28:19,  1.18it/s]


In [8]:
for f in tqdm(bucket.objects.filter(Prefix='PART2/DATA/CHANNEL0/WAVE')):
    filename = os.path.join('part2-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

1031it [43:53,  2.55s/it]


In [11]:
zips = sorted(glob('part2-audio/*.zip'))
for f in tqdm(zips):
    os.system(f'unzip -o {f} -d part2-audio > /dev/null')
    os.remove(f)

100%|██████████| 1031/1031 [30:39<00:00,  1.78s/it] 


In [13]:
import soundfile as sf
from glob import glob

In [14]:
texts = sorted(glob('part2/*'))
len(texts)

2003

In [15]:
filtered = []

for f in tqdm(texts):
    with open(f) as fopen:
        t = fopen.read().split('\n')

    for t_ in t:
        if not len(t_):
            continue
        splitted = t_.split('\t')
        l = unidecode(splitted[0])
        if not len(l):
            continue
        r = unidecode(splitted[1].strip())
        
        speaker = f'SPEAKER{l[1:-4]}'
        session = l[-4]
        filename = os.path.join('part2-audio', speaker, f'SESSION{session}', f'{l}.WAV')
        if not os.path.exists(filename):
            continue
            
        d = {
            'id': l,
            'text': r,
            'filename': filename
        }
        filtered.append(d)
        
len(filtered)

100%|██████████| 2003/2003 [00:05<00:00, 375.36it/s]


826469

In [16]:
filtered[-1]

{'id': '035001890',
 'text': 'Can you recommend me a restaurant near Greenfield Drive?',
 'filename': 'part2-audio/SPEAKER3500/SESSION1/035001890.WAV'}

In [17]:
ipd.Audio(filtered[-1]['filename'])

In [18]:
!mkdir part2-mp3

In [19]:
import mp

In [20]:
def loop(filtered):
    filtered, _ = filtered
    selected = []
    for f in tqdm(filtered):
        try:
            y, sr = sf.read(f['filename'])
            new_filename = f"part2-mp3/{f['id']}.mp3"
            sf.write(new_filename, y, sr)
            f['new_filename'] = new_filename
            selected.append(f)
        except:
            pass
    return selected

In [21]:
results = mp.multiprocessing(filtered, loop, cores = 20)

  4%|▍         | 1631/41323 [03:33<16:14:46,  1.47s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 24%|██▍       | 9846/41323 [21:00<32:24, 16.19it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 89%|████████▊ | 36584/41323 [1:42:42<05:33, 14.21it/s]   

In [24]:
len(results)

826469

In [38]:
results[1]

{'id': '020010002',
 'text': 'DC, Asos, and FairPrice.',
 'filename': 'part2-audio/SPEAKER2001/SESSION0/020010002.WAV',
 'new_filename': 'part2-mp3/020010002.mp3'}

In [40]:
ipd.Audio(results[1]['new_filename'])

In [26]:
import json

with open('imda-part2.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 826469/826469 [00:02<00:00, 286861.93it/s]


In [27]:
from huggingface_hub import HfApi
api = HfApi()

In [28]:
api.upload_file(
    path_or_fileobj='imda-part2.jsonl',
    path_in_repo='imda-part2.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-part2.jsonl:   0%|          | 0.00/148M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-part2.jsonl'

In [29]:
# !sudo apt install p7zip-full p7zip-rar -y

In [32]:
!7z -v5g a part2-mp3.7z part2-mp3 > /dev/null

In [33]:
for f in glob('part2-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part2-mp3.7z.004


part2-mp3.7z.004:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

part2-mp3.7z.002


part2-mp3.7z.002:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part2-mp3.7z.003


part2-mp3.7z.003:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part2-mp3.7z.001


part2-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [34]:
total_len = 0
for r in tqdm(results):
    y, sr = sf.read(r['filename'])
    total_len += len(y) / sr

100%|██████████| 826469/826469 [05:13<00:00, 2637.92it/s]


In [35]:
total_len / 60 / 60

1052.1777120312486

In [36]:
!rm -rf part2-audio